# Transformt Data with the constrain of 1 seconds , breaks and encoding for NLP

## Info before preprocesing

In [12]:
import pickle
import numpy as np
from Vectorisation import Vectorisation
from Config import Config

In [13]:
# I manually move the data (locally) inside this folder 
with open('./ml4science_data.pkl', 'rb') as fp:
    dl = pickle.load(fp)

In [14]:
print(dl.keys())
dl['sequences'][0].keys()

dict_keys(['sequences', 'index', 'available_demographics'])


dict_keys(['path', 'length', 'learner_id', 'raw_sequence', 'raw_begin', 'raw_end', 'last_timestamp', 'permutation', 'gender', 'year', 'language', 'begin', 'end', 'break_sequence', 'sequence', 'label', '1hot-sequence', 'nobreak', 'nobreak-1hot-sequence', 'stratifier_column'])

In [15]:
for i in range(15):
    print('begin {} - end {}: {}'.format(dl['sequences'][0]['begin'][i], dl['sequences'][0]['end'][i], dl['sequences'][0]['sequence'][i]))

begin 0 - end 7.889: [0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3440621047581669]
begin 7.889 - end 83.03: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0]
begin 83.03 - end 86.28: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14174189890531658]
begin 86.28 - end 86.33: [0.0, 0.0, 0.0, 1.0, 0.006079027355622759, 0.0, 0.0, 0.0, 0.0, 0.0]
begin 87.278 - end 90.562: [0.0, 0.0, 0.0, 1.0, 0.3992705167173245, 0.0, 0.0, 0.0, 0.0, 0.0]
begin 90.562 - end 99.597: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3940424789567799]
begin 99.597 - end 100.32: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.1239499399965717, 0.0, 0.0, 0.0]
begin 101.267 - end 107.071: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0772414527355239, 0.0]
begin 108.989 - end 114.653: [0.0, 0.0, 0.0, 1.0, 0.6886322188449855, 0.0, 0.0, 0.0, 0.0, 0.0]
begin 115.424 - end 115.935: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.08760500600034259, 0.0, 0.0, 0.0]
begin 117.488 - end 135.421: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.23865798

In [16]:
student_seq = [len(dl['sequences'][i]['sequence']) for i in range(len(dl['sequences']))]

print(np.mean(student_seq), '+-', np.std(student_seq))
print(np.min(student_seq))
print(np.max(student_seq))

100.31496062992126 +- 102.75095652848267
13
819


## Repair FAKE data

In [17]:
'''for i in range(len(dl['sequences'])):
    for j in range(len(dl['sequences'][i]['sequence'])): 
        non_zero_count = np.count_nonzero(dl['sequences'][i]['sequence'][j])
        if non_zero_count != 2:
            #add randomly a one in of of the four first values of the sequence
            dl['sequences'][i]['sequence'][j][np.random.randint(0,4)] = 1'''

"for i in range(len(dl['sequences'])):\n    for j in range(len(dl['sequences'][i]['sequence'])): \n        non_zero_count = np.count_nonzero(dl['sequences'][i]['sequence'][j])\n        if non_zero_count != 2:\n            #add randomly a one in of of the four first values of the sequence\n            dl['sequences'][i]['sequence'][j][np.random.randint(0,4)] = 1"

In [18]:
for i in range(len(dl['sequences'])):
    for j in range(len(dl['sequences'][i]['sequence'])): 
        non_zero_count = np.count_nonzero(dl['sequences'][i]['sequence'][j])
        if non_zero_count != 2:
            print(f"Vector at student {i}, sequence {j} has {non_zero_count} non-zero values")
            non_zero_positions = np.nonzero(dl['sequences'][i]['sequence'][j])[0]
            print(f"Positions of non-zero values: {non_zero_positions}")

# Manual check
print(dl['sequences'][20]['sequence'][1])
print(dl['sequences'][20]['sequence'][2])
print(dl['sequences'][120]['sequence'][5])
print(dl['sequences'][120]['sequence'][6])

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3564261315115266]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34055533828705625]


In [19]:
# save data in a pkl file
#with open('./ml4science_data_fake_repaired.pkl', 'wb') as fp:
#    pickle.dump(dl, fp)

## Test for create Vocabulary

In [21]:
config = Config()

token_dict={'[PAD]': 1, '[SEP]': 2, '[MASK]': 3} # PAD -> 0 is mush better
tf = Vectorisation(config=config, special_token_dict=token_dict, sep=15)

encoded = tf.encode_vocabulary(dl)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
for i in range(15): #range(len(dl['sequences'][0]['begin'])):
    print(
        'begin {} - end {}: {}'.format
        (encoded['sequences'][0]['begin'][i],
        encoded['sequences'][0]['end'][i],
        encoded['sequences'][0]['sequence'][i])
    )

begin 0 - end 7.889: 27
begin 7.889 - end 83.03: 2
begin 83.03 - end 86.28: 27
begin 86.28 - end 86.33: 22
begin 87.278 - end 90.562: 22
begin 90.562 - end 99.597: 27
begin 99.597 - end 100.32: 24
begin 101.267 - end 107.071: 26
begin 108.989 - end 114.653: 22
begin 115.424 - end 115.935: 24
begin 117.488 - end 135.421: 2
begin 136.702 - end 144.927: 22
begin 144.927 - end 147.995: 27
begin 147.995 - end 149.956: 22
begin 150.243 - end 150.934: 22


In [ ]:
decoded = tf.decode_vocabulary(encoded)

for i in range(15): #range(len(dl['sequences'][0]['begin'])):
    print(
        'begin {} - end {}: {}'.format
        (decoded['sequences'][0]['begin'][i],
        decoded['sequences'][0]['end'][i],
        decoded['sequences'][0]['sequence'][i])
    )

begin 0 - end 7.889: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
begin 7.889 - end 83.03: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
begin 83.03 - end 86.28: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
begin 86.28 - end 86.33: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
begin 87.278 - end 90.562: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
begin 90.562 - end 99.597: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
begin 99.597 - end 100.32: [0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
begin 101.267 - end 107.071: [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
begin 108.989 - end 114.653: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
begin 115.424 - end 115.935: [0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
begin 117.488 - end 135.421: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
begin 136.702 - end 144.927: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
begin 144.927 - end 147.995: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
begin 147.995 - end 149.956: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
begin 150.243 - end 150.934: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
